In [6]:
import numpy as np
import os
from pathlib import Path

from e.src.lib.placeholders import (
    ParametrosFisicos,
    NpuntosDireccion,
    ParametrosGeometricos,
    ParametrosComputacionales
)

from e.src.lib.constants import Rutas

In [7]:
print(Rutas.RUTA_PAQUETE)

/home/ramanujan/e3/e


In [8]:
temperaturas = ParametrosFisicos(T0=0, T1=50)
Npuntos = NpuntosDireccion(Nx=10, Ny=10)
SCparams = ParametrosGeometricos(R=1.0)
CompParams = ParametrosComputacionales(max_iteraciones=int(10e4), tolerancia=10e-6)

In [9]:
print(temperaturas)
print(Npuntos)
print(SCparams)
print(CompParams)

ParametrosFisicos(T0=0, T1=50)
NpuntosDireccion(Nx=10, Ny=10)
ParametrosGeometricos(R=1.0)
ParametrosComputacionales(max_iteraciones=100000, tolerancia=1e-05)


In [32]:
u = np.zeros(shape=(Npuntos.Nx+1, Npuntos.Ny+1))

def respetar_condiciones_frontera(u: np.ndarray):
    
    # Dirichlet.
    u[:, 0] = temperaturas.T0   # En theta = 0
    u[:, -1] = temperaturas.T0  # En theta = pi
    u[-1, :] = temperaturas.T1  # En r = 1
    
    # Neumann.
    u[0, :] = u[1, :]
    
    return u

# Condiciones iniciales.
u[0] = 0
u[-1] = 50
u = respetar_condiciones_frontera(u)

In [20]:
u

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.]])

In [21]:
def metodo_sor(
    u,
    Nx: int,
    Ny: int,
    Dx: float,
    Dy: float,
    omega: float,
    maximas_iteraciones: int,
    tolerancia: float,
):
    # Iteraciones del método SOR
    for it in range(maximas_iteraciones):
        u_old = np.copy(u)
        
        # Actualizar solo los puntos interiores de la malla
        for i in range(1, Nx - 1):  # Excluir las condiciones en r = 0 y r = 1
            r_i = i * Dx
            for j in range(1, Ny - 1):  # Excluir las condiciones en theta = 0 y theta = pi
                u_new = (1 / (2 * (1/Dx**2 + 1/(r_i**2 * Dy**2)))) * (
                    (u[i+1, j] + u[i-1, j]) / Dx**2 + 
                    (1 / (r_i**2)) * (u[i, j+1] + u[i, j-1]) / Dy**2
                )
                
                # Sobrerrelajación
                u[i, j] = (1 - omega) * u[i, j] + omega * u_new
        
        u = respetar_condiciones_frontera(u)
        
        # Criterio de convergencia.
        diff = np.max(np.abs(u - u_old))
        if diff < tolerancia:
            print(f'Convergencia alcanzada en {it} iteraciones.')
            break
    else:
        print('No se alcanzó la convergencia.')

    return u

In [ ]:
# Dx : 0, R/4, 2R/4, ...
# Dy : 0, pi/4, 2pi/4, ...

solution = metodo_sor(
    Nx=Npuntos.Nx, 
    Ny=Npuntos.Ny,
    Dx=SCparams.R/Npuntos.Nx, 00
    Dy=Npuntos.Ny,
    omega=1.5,
    tolerancia=CompParams.tolerancia,
    maximas_iteraciones=CompParams.max_iteraciones
)

In [11]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros del problema
R = 1.0    # Radio de la lámina
T0 = 0.0   # Temperatura en el diámetro (y = 0)
T1 = 50.0  # Temperatura en el borde circular
N = 50     # Número de divisiones en x y y
max_iter = 10000  # Máximo número de iteraciones
tolerance = 1e-6  # Tolerancia para la convergencia

# Malla
x = np.linspace(-R, R, N + 1)
y = np.linspace(0, R, N + 1)
dx = x[1] - x[0]
dy = y[1] - y[0]
X, Y = np.meshgrid(x, y, indexing='ij')

# Máscara para el semicírculo
mask = X**2 + Y**2 <= R**2

# Condiciones de frontera
diameter = mask & np.isclose(Y, 0)
circular_arc = mask & np.isclose(X**2 + Y**2, R**2)

# Inicialización de la matriz de temperaturas
u = np.zeros((N + 1, N + 1))
u[diameter] = T0        # Diámetro a T0
u[circular_arc] = T1    # Borde circular a T1

# Iteración
for iteration in range(max_iter):
    error = 0.0
    u_old = u.copy()
    for i in range(1, N):
        for j in range(1, N):
            if mask[i, j] and not diameter[i, j] and not circular_arc[i, j]:
                sum_neighbors = 0.0
                count_neighbors = 0

                # Vecino derecho (i+1, j)
                if i + 1 <= N:
                    if mask[i+1, j]:
                        sum_neighbors += u[i+1, j]
                        count_neighbors += 1
                    elif circular_arc[i+1, j]:
                        sum_neighbors += T1
                        count_neighbors += 1
                    elif diameter[i+1, j]:
                        sum_neighbors += T0
                        count_neighbors += 1

                # Vecino izquierdo (i-1, j)
                if i - 1 >= 0:
                    if mask[i-1, j]:
                        sum_neighbors += u[i-1, j]
                        count_neighbors += 1
                    elif circular_arc[i-1, j]:
                        sum_neighbors += T1
                        count_neighbors += 1
                    elif diameter[i-1, j]:
                        sum_neighbors += T0
                        count_neighbors += 1

                # Vecino superior (i, j+1)
                if j + 1 <= N:
                    if mask[i, j+1]:
                        sum_neighbors += u[i, j+1]
                        count_neighbors += 1
                    elif circular_arc[i, j+1]:
                        sum_neighbors += T1
                        count_neighbors += 1
                    elif diameter[i, j+1]:
                        sum_neighbors += T0
                        count_neighbors += 1

                # Vecino inferior (i, j-1)
                if j - 1 >= 0:
                    if mask[i, j-1]:
                        sum_neighbors += u[i, j-1]
                        count_neighbors += 1
                    elif circular_arc[i, j-1]:
                        sum_neighbors += T1
                        count_neighbors += 1
                    elif diameter[i, j-1]:
                        sum_neighbors += T0
                        count_neighbors += 1

                # Actualizar u[i, j]
                if count_neighbors > 0:
                    old_uij = u[i, j]
                    u[i, j] = sum_neighbors / count_neighbors
                    diff = abs(u[i, j] - old_uij)
                    if diff > error:
                        error = diff

    # Aplicar condiciones de frontera nuevamente
    u[diameter] = T0
    u[circular_arc] = T1

    if error < tolerance:
        print(f'Convergencia alcanzada en {iteration + 1} iteraciones.')
        break
else:
    print('No se alcanzó la convergencia.')

# Visualización
plt.figure(figsize=(8, 6))
plt.contourf(X, Y, u, levels=50, cmap='hot')
plt.colorbar(label='Temperatura (°C)')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Distribución de temperatura en coordenadas cartesianas')
plt.show()


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Analytical solution
def analytical_solution(x):
    import math
    cos1 = np.cos(1)
    return (np.sin(x)/cos1) - x

def fem_solver_linear(n_elements):
    degree = 1  # Linear interpolation
    n_nodes = n_elements * degree + 1 - (degree - 1)
    h = 1.0 / n_elements
    x = np.linspace(0, 1, n_nodes)

    K = np.zeros((n_nodes, n_nodes))
    f = np.zeros(n_nodes)

    # Gauss points and weights for 2-point quadrature
    gauss_pts = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
    gauss_wts = np.array([1.0, 1.0])

    for e in range(n_elements):
        nodes = [e, e+1]
        xi = x[nodes]
        h_e = xi[1] - xi[0]

        Ke = np.zeros((2,2))
        fe = np.zeros(2)

        for k in range(len(gauss_pts)):
            xi_gp = gauss_pts[k]
            w_gp = gauss_wts[k]

            # Map from reference [-1,1] to element [x_i, x_{i+1}]
            x_gp = ((xi[1] + xi[0]) / 2) + xi_gp * (h_e / 2)
            dx_dxi = h_e / 2

            # Shape functions and derivatives
            N = np.array([ (1 - xi_gp)/2, (1 + xi_gp)/2 ])
            dN_dxi = np.array([ -0.5, 0.5 ])
            dN_dx = dN_dxi * (2 / h_e)  # Chain rule

            # Assemble element matrices
            Ke += (np.outer(dN_dx, dN_dx) + np.outer(N, N)) * w_gp * dx_dxi
            fe += -N * x_gp * w_gp * dx_dxi

        # Assemble into global matrices
        for i in range(2):
            for j in range(2):
                K[nodes[i], nodes[j]] += Ke[i, j]
            f[nodes[i]] += fe[i]

    # Apply Dirichlet BC at x=0
    K = K[1:, 1:]
    f = f[1:]

    # Solve the system
    u = np.linalg.solve(K, f)

    # Insert boundary condition
    u = np.insert(u, 0, 0.0)

    return x, u

# Solve using 4 elements
x, u = fem_solver_linear(n_elements=4)
u_exact = analytical_solution(x)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(x, u, 'bo-', label='FEM Solution (4 elements)')
plt.plot(x, u_exact, 'r--', label='Analytical Solution')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.legend()
plt.title('Solution with 4 Elements and Linear Interpolation')
plt.grid(True)
plt.show()

# Error analysis
error_norm = np.linalg.norm(u - u_exact) / np.linalg.norm(u_exact)
print('Relative error norm (4 elements, linear):', error_norm)


In [ ]:
# Solve using 8 elements
x, u = fem_solver_linear(n_elements=8)
u_exact = analytical_solution(x)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(x, u, 'bo-', label='FEM Solution (8 elements)')
plt.plot(x, u_exact, 'r--', label='Analytical Solution')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.legend()
plt.title('Solution with 8 Elements and Linear Interpolation')
plt.grid(True)
plt.show()

# Error analysis
error_norm = np.linalg.norm(u - u_exact) / np.linalg.norm(u_exact)
print('Relative error norm (8 elements, linear):', error_norm)


In [ ]:
def fem_solver_quadratic(n_elements):
    degree = 2  # Quadratic interpolation
    n_nodes = n_elements * degree + 1 - (degree - 1)
    h = 1.0 / n_elements
    x = np.linspace(0, 1, n_nodes)

    K = np.zeros((n_nodes, n_nodes))
    f = np.zeros(n_nodes)

    # Gauss points and weights for 3-point quadrature
    gauss_pts = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])
    gauss_wts = np.array([5/9, 8/9, 5/9])

    for e in range(n_elements):
        nodes = [e*2, e*2+1, e*2+2]
        xi = x[nodes]
        h_e = xi[2] - xi[0]

        Ke = np.zeros((3,3))
        fe = np.zeros(3)

        for k in range(len(gauss_pts)):
            xi_gp = gauss_pts[k]
            w_gp = gauss_wts[k]

            # Map from reference [-1,1] to element [x_i, x_{i+1}]
            x_gp = ((xi[2] + xi[0]) / 2) + xi_gp * (h_e / 2)
            dx_dxi = h_e / 2

            # Shape functions
            N = np.array([
                xi_gp * (xi_gp - 1) / 2,
                (1 - xi_gp**2),
                xi_gp * (xi_gp + 1) / 2
            ])
            dN_dxi = np.array([
                xi_gp - 0.5,
                -2 * xi_gp,
                xi_gp + 0.5
            ])
            dN_dx = dN_dxi * (2 / h_e)  # Chain rule

            # Assemble element matrices
            Ke += (np.outer(dN_dx, dN_dx) + np.outer(N, N)) * w_gp * dx_dxi
            fe += -N * x_gp * w_gp * dx_dxi

        # Assemble into global matrices
        for i in range(3):
            for j in range(3):
                K[nodes[i], nodes[j]] += Ke[i, j]
            f[nodes[i]] += fe[i]

    # Apply Dirichlet BC at x=0
    K = K[1:, 1:]
    f = f[1:]

    # Solve the system
    u = np.linalg.solve(K, f)

    # Insert boundary condition
    u = np.insert(u, 0, 0.0)

    return x, u

# Solve using 2 elements
x, u = fem_solver_quadratic(n_elements=2)
u_exact = analytical_solution(x)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(x, u, 'bo-', label='FEM Solution (2 elements)')
plt.plot(x, u_exact, 'r--', label='Analytical Solution')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.legend()
plt.title('Solution with 2 Elements and Quadratic Interpolation')
plt.grid(True)
plt.show()

# Error analysis
error_norm = np.linalg.norm(u - u_exact) / np.linalg.norm(u_exact)
print('Relative error norm (2 elements, quadratic):', error_norm)


In [ ]:
# Solve using 4 elements
x, u = fem_solver_quadratic(n_elements=4)
u_exact = analytical_solution(x)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(x, u, 'bo-', label='FEM Solution (4 elements)')
plt.plot(x, u_exact, 'r--', label='Analytical Solution')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.legend()
plt.title('Solution with 4 Elements and Quadratic Interpolation')
plt.grid(True)
plt.show()

# Error analysis
error_norm = np.linalg.norm(u - u_exact) / np.linalg.norm(u_exact)
print('Relative error norm (4 elements, quadratic):', error_norm)
